# Toy Case

This case employs a simple reaction `A > B` performed in a batch reactor to introduce the provided modelling environment.  

<div><center><img src="https://upload.wikimedia.org/wikipedia/commons/1/15/Batch_reactor_STR.svg" width="150"/></center></div>

The mass balance equation for batch reactor is
$$\frac{dc}{dt} = r_r$$  
The reaction rate constant of `A > B` follows Arrhenius equation
$$k = A\cdot\exp{(-\frac{E_a}{RT})}$$
We reparameterise Arrhenius with reference to 20 oC
$$k = k_{ref}\cdot\exp{(-\frac{E_a}{R}(\frac{1}{T} - \frac{1}{293.15}))}$$
The reaction rate follows power law with respect to concentrations of reactants
$$r_r = k\prod c^n$$

In [1]:
# Import required python libraries
# Run this cell to set up the modelling environment
import pandas as pd
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

# Define the phenomenon dictionary to define the model structure
# A dictionary is a key-value mapping. For more info, please look 
# at https://docs.python.org/3/tutorial/datastructures.html#dictionaries
# Mass accumulation and flow pattern phenomenons are given in string, 
# Mass transport phenomenons are collected in a list, though it's empty here.
phenos = {
    "Mass accumulation":    "Batch",
    "Flow pattern":         "Well_Mixed",
    "Mass transport":       [],
}

# Import the toy case study which we'll work on in this notebook
from cases.toy_case import ToyCase
toy_case = ToyCase(phenos)

#### Parameter Setup
As modelling parameters are associated with different dimensions (solid, liquid, stream, reaction, and species)  

All parameters are indicated as a dictionary as  
```
{
    "param":    "xxx", 
    "solid":    "xxx", 
    "gas":      "xxx", 
    "stream":   "xxx",
    "reaction": "xxx",
    "species":  "xxx"
}
```
in this provided modelling environment.  

Example parameter dictionaries:  
> - **Operation parameters**
>   - {"param": "Temperature"}
>   - {"Param": "Concentration", "stream": "Stream_1", "species": "A"}  
> Concentration is associated with stream and species  
> - **Kinetics parameters**
>   - {"param": "Activation_Energy", "stream": "Stream_1", "reaction": "A > B"}
>   - {"param": "Referenced_Reaction_Rate_Constant", "stream": "Stream_1", "reaction": "A > B"}  
> Activation_Energy and Referenced_Reaction_Rate_Constant are associated with stream and reaction

#### Unit Setup

Parameter setup can be found in the method `toy_case.var2unit()`

| Parameter                         | Unit   |
| --------------------------------- | ------ |
| Referenced_Reaction_Rate_Constant |        |
| Activation_Energy                 | kJ/mol |
| Temperature                       | oC     |
| Concentration                     | mol/L  |
| Batch_Time                        | min    |

### Model Simulation
- Print model kinetics parameters and operation parameters
- Set operation condition example and plot the concentration curves against operation time
- Plot B concentration profiles at varied temperatures
- Plot B concentration profiles at varied initial A concentrations

In [2]:
# List reaction parameters
print("Reaction parameters:")
for reaction_param in toy_case.reaction_params():
    print(reaction_param)

Reaction parameters:
({'param': 'Activation_Energy', 'stream': 'Batch_Feed', 'reaction': 'A > B'}, 80.0)
({'param': 'Referenced_Reaction_Rate_Constant', 'stream': 'Batch_Feed', 'reaction': 'A > B'}, 0.0001)
({'param': 'Stoichiometric_Coefficient', 'reaction': 'A > B', 'species': 'A'}, -1)
({'param': 'Stoichiometric_Coefficient', 'reaction': 'A > B', 'species': 'B'}, 1)
({'param': 'Partial_Order', 'reaction': 'A > B', 'species': 'A'}, 1)


In [3]:
# List operation parameters
print("Operation parameters:")
for operation_param in toy_case.operation_params():
    print(operation_param)

Operation parameters:
({'param': 'Concentration', 'stream': 'Batch_Feed', 'species': 'A'}, None)
({'param': 'Temperature'}, None)
({'param': 'Batch_Time'}, None)


In [4]:
# Model simulation and plot concentration profiles
# A concentration   [A] = 1 mol/L
# Temperature       T = 50.0 oC
# Batch time        t_b = 60.0 min
operation_params = [
    ({"param": "Concentration", "stream": "Batch_Feed", "species": "A"}, 1.0), # mol/L
    ({"param": "Temperature"}, 50.0), # oC
    ({"param": "Batch_Time"}, 60.0), # min
]
toy_case.plot_simulation_profiles(operation_params)

In [5]:
# B concentration profiles under vared temperatures
# Set temperature to a list of values: 25.0, 50.0, 75.0, 100.0
operation_params = [
    ({"param": "Concentration", "stream": "Batch_Feed", "species": "A"}, 1.0), # mol/L
    ({"param": "Temperature"}, [25.0, 50.0, 75.0, 100.0]), # oC
    ({"param": "Batch_Time"}, 60.0), # min
]
toy_case.plot_product_profile_with_temperatures(operation_params)

In [6]:
# B concentration profiles under vared A concentrations
# Set A concentration to a list of values: [0.5, 1.0, 1.5, 2.0]
operation_params = [
    ({"param": "Concentration", "stream": "Batch_Feed", "species": "A"}, [0.5, 1.0, 1.5, 2.0]), # mol/L
    ({"param": "Temperature"}, 50.0), # oC
    ({"param": "Batch_Time"}, 60.0), # min
]
toy_case.plot_product_profile_with_A_concs(operation_params)

### Model Calibration
How to design experiments to determine kinetics?
In this notebook, we use a simple grid-based experimental approach.  
Recall that  
$k = k_{ref}\cdot\exp{(-\frac{E_a}{R}(\frac{1}{T} - \frac{1}{293.15}))}$, $r_r = k\prod c^n$  
By varying residence time and initial A concentration, we can calibrate $k$ for each temperature  
By varying temperature, we can calibrate the Activation Energy $E_a$

In [7]:
# Print column names for operation parameters
print("Column names for operation parameters:")
for name_ind in toy_case.operation_name2ind():
    print(name_ind)

Column names for operation parameters:
('A_conc', {'param': 'Concentration', 'stream': 'Batch_Feed', 'species': 'A'})
('temp', {'param': 'Temperature'})
('t_b', {'param': 'Batch_Time'})


### Prediction error minimisation
Assume these data are obtained from experiments. Then, the model is fitted by minimising the prediction error on these operation conditions  
$Error = Mean((c_{measured,B} - c_{predicted,B})^2)$  
The quasi-Newton algorithm `L-BFGS-B` is used to minimise this error  
Find original codes of the `calibrate` method in `cases/toy_case.py` file

In [8]:
# First, we design grid experiment points and calibrate k_ref at referenced temperature (25 oC)
# Run simulation to mimic experiment data
exp_data1 = {
    "A_conc": [0.25, 0.25, 0.75, 0.75],
    "temp":   [  25,   25,   25,   25],
    "t_b":    [  30,   60,   30,   60],
}
exp_dataset1 = pd.DataFrame(exp_data1)
exp_dataset1 = toy_case.run_dataset(exp_dataset1)

# Define parameter bound for calibration
cal_param_bounds = [
    (
        {
            "param": "Referenced_Reaction_Rate_Constant", 
            "stream": "Batch_Feed", 
            "reaction": "A > B"
        },
        (1e-6, 1e-3)
    )
]

# Calbrate referenced reaction rate constant
calibrated_kref = toy_case.calibrate(cal_param_bounds, exp_dataset1)[0]
print(calibrated_kref)

({'param': 'Referenced_Reaction_Rate_Constant', 'stream': 'Batch_Feed', 'reaction': 'A > B'}, 0.0001)


In [9]:
# Then, we design grid experiment points for 50 and 75 oC to calibrate activation energy
# Run simulation to mimic experiment data
exp_data2 = {
    "A_conc": [0.25, 0.25, 0.75, 0.75, 0.25, 0.25, 0.75, 0.75],
    "temp":   [  50,   50,   50,   50,   75,   75,   75,   75],
    "t_b":    [  30,   60,   30,   60,   30,   60,   30,   60],
}
exp_dataset2 = pd.DataFrame(exp_data2)
exp_dataset2 = toy_case.run_dataset(exp_dataset2)

# Define parameter bound for calibration
cal_param_bounds = [
    (
        {
            "param": "Referenced_Reaction_Rate_Constant", 
            "stream": "Batch_Feed", 
            "reaction": "A > B"
        },
        (1e-4, 1e-4)
    ), 
    (
        {
            "param": "Activation_Energy", 
            "stream": "Batch_Feed", 
            "reaction": "A > B"
        },
        (20, 100)
    )
]

# Calbrate activation energy
calibrated_params = toy_case.calibrate(cal_param_bounds, exp_dataset2)
for calibrated_param in calibrated_params:
    print(calibrated_param)

({'param': 'Activation_Energy', 'stream': 'Batch_Feed', 'reaction': 'A > B'}, 60.140533)
({'param': 'Referenced_Reaction_Rate_Constant', 'stream': 'Batch_Feed', 'reaction': 'A > B'}, 0.0001)
